# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), August 25, 2018**

Let's get started by loading the `DataFrames` package.

In [1]:
using DataFrames, Random

## Constructors and conversion

### Constructors

In this section, you'll see many ways to create a `DataFrame` using the `DataFrame()` constructor.

First, we could create an empty DataFrame,

In [2]:
DataFrame() # empty DataFrame

""


Or we could call the constructor using keyword arguments to add columns to the `DataFrame`.

In [3]:
DataFrame(A=1:3, B=rand(3), C=randstring.([3,3,3]))

,A,B,C
1,1,0.410792,8tB
2,2,0.421197,3d2
3,3,0.14108,exY


We can create a `DataFrame` from a dictionary, in which case keys from the dictionary will be sorted to create the `DataFrame` columns.

In [4]:
x = Dict("A" => [1,2], "B" => [true, false], "C" => ['a', 'b'])
DataFrame(x)

,A,B,C
1,1,true,'a'
2,2,false,'b'


Rather than explicitly creating a dictionary first, as above, we could pass `DataFrame` arguments with the syntax of dictionary key-value pairs. 

Note that in this case, we use symbols to denote the column names and arguments are not sorted. For example, `:A`, the symbol, produces `A`, the name of the first column here:

In [5]:
DataFrame(:A => [1,2], :B => [true, false], :C => ['a', 'b'])

,A,B,C
1,1,true,'a'
2,2,false,'b'


Here we create a `DataFrame` from a vector of vectors, and each vector becomes a column.

In [6]:
DataFrame([rand(3) for i in 1:3])

,x1,x2,x3
1,0.388437,0.798451,0.122055
2,0.410617,0.960645,0.554145
3,0.23046,0.481033,0.239433


 For now we can construct a single `DataFrame` from a `Vector` of atoms, creating a `DataFrame` with a single row. In future releases of DataFrames.jl, this will throw an error.

In [7]:
DataFrame(rand(3))

┌ Warning: passing columns argument with non-AbstractVector entries is deprecated
│   caller = top-level scope at none:0
└ @ Core none:0


,x1,x2,x3
1,0.595378,0.747479,0.0775951


Instead use a transposed vector if you have a vector of atoms (in this way you effectively pass a two dimensional array to the constructor which is supported).

In [8]:
DataFrame(permutedims([1, 2, 3]))

,x1,x2,x3
1,1,2,3


Pass a second argument to give the columns names.

In [9]:
DataFrame([1:3, 4:6, 7:9], [:A, :B, :C])

,A,B,C
1,1,4,7
2,2,5,8
3,3,6,9


Here we create a `DataFrame` from a matrix,

In [10]:
DataFrame(rand(3,4))

,x1,x2,x3,x4
1,0.0261509,0.212892,0.203718,0.389656
2,0.788717,0.339639,0.122765,0.675641
3,0.0375159,0.705161,0.918844,0.667094


and here we do the same but also pass column names.

In [11]:
DataFrame(rand(3,4), Symbol.('a':'d'))

,a,b,c,d
1,0.311832,0.554788,0.328902,0.462667
2,0.84578,0.166949,0.51255,0.0170572
3,0.2838,0.353377,0.883708,0.61586


We can also construct an uninitialized DataFrame.

Here we pass column types, names and number of rows; we get `missing` in column :C because `Any >: Missing`.

In [12]:
DataFrame([Int, Float64, Any], [:A, :B, :C], 1)

,A,B,C
1,17179869187,6.89903e-310,missing


Here we create a `DataFrame` where `:C` is #undef

In [13]:
DataFrame([Int, Float64, String], [:A, :B, :C], 1)

,A,B,C
1,139930560114288,6.91349e-310,#undef


To initialize a `DataFrame` with column names, but no rows use

In [14]:
DataFrame([Int, Float64, String], [:A, :B, :C], 0) 

,A,B,C


or

In [15]:
DataFrame(A=Int[], B=Float64[], C=String[])

,A,B,C


This syntax gives us a quick way to create homogenous `DataFrame`.

In [16]:
DataFrame(Int, 3, 5)

,x1,x2,x3,x4,x5
1,139930560077832,139929701389136,139930560081472,139930560077832,139929701389584
2,139929701389584,139930562726096,139930560081536,139929701389584,139929711513968
3,139929711513600,139930562724416,139930560081600,139929711513840,0


This example is similar, but has nonhomogenous columns.

In [17]:
DataFrame([Int, Float64], 4)

,x1,x2
1,139930719207184,6.9135e-310
2,139930560082432,6.91349e-310
3,139930560082368,6.91349e-310
4,139930562724416,6.91349e-310


Finally, we can create a `DataFrame` by copying an existing `DataFrame`.

Note that `copy` creates a shallow copy.

In [18]:
x

Dict{String,Array{T,1} where T} with 3 entries:
  "B" => Bool[true, false]
  "A" => [1, 2]
  "C" => ['a', 'b']

In [19]:
x = DataFrame(a=1:2, b='a':'b')
y = copy(x)
(x === y), isequal(x, y)

(false, true)

### Conversion to a matrix

Let's start by creating a `DataFrame` with two rows and two columns.

In [20]:
x = DataFrame(x=1:2, y=["A", "B"])

,x,y
1,1,A
2,2,B


We can create a matrix by passing this `DataFrame` to `Matrix`.

In [21]:
Matrix(x)

2×2 Array{Any,2}:
 1  "A"
 2  "B"

This would work even if the `DataFrame` had some `missing`s:

In [22]:
x = DataFrame(x=1:2, y=[missing,"B"])

,x,y
1,1,missing
2,2,B


In [23]:
Matrix(x)

2×2 Array{Any,2}:
 1  missing
 2  "B"    

In the two previous matrix examples, Julia created matrices with elements of type `Any`. We can see more clearly that the type of matrix is inferred when we pass, for example, a `DataFrame` of integers to `Matrix`, creating a 2D `Array` of `Int64`s:

In [24]:
x = DataFrame(x=1:2, y=3:4)

,x,y
1,1,3
2,2,4


In [25]:
Matrix(x)

2×2 Array{Int64,2}:
 1  3
 2  4

In this next example, Julia correctly identifies that `Union` is needed to express the type of the resulting `Matrix` (which contains `missing`s).

In [26]:
x = DataFrame(x=1:2, y=[missing,4])

,x,y
1,1,missing
2,2,4


In [27]:
Matrix(x)

2×2 Array{Union{Missing, Int64},2}:
 1   missing
 2  4       

Note that we can't force a conversion of `missing` values to `Int`s!

In [28]:
Matrix{Int}(x)

LoadError: cannot convert a DataFrame containing missing values to array (found for column y)

### Handling of duplicate column names

We can pass the `makeunique` keyword argument to allow passing duplicate names (they get deduplicated)

In [29]:
df = DataFrame(:a=>1, :a=>2, :a_1=>3; makeunique=true)

,a,a_2,a_1
1,1,2,3


Otherwise, duplicates will not be allowed in the future.

In [30]:
df = DataFrame(:a=>1, :a=>2, :a_1=>3)

┌ Warning: Duplicate variable names are deprecated: pass makeunique=true to add a suffix automatically.
│   caller = ip:0x0
└ @ Core :-1


,a,a_2,a_1
1,1,2,3


Finallly observe that `nothing` is not printed when displaying a `DataFrame`:

In [31]:
DataFrame(x=[1, nothing], y=[nothing, "a"])

,x,y
1,1,
2,,a
